In [17]:
from __future__ import division
import findspark
findspark.init()
from pyspark import SparkContext, SQLContext, SparkConf
from pyspark.storagelevel import StorageLevel
import numpy as np
from decimal import Decimal
import matplotlib.pyplot as plt; plt.rcdefaults()

In [18]:
sc = SparkContext.getOrCreate();

Use when you are coding on notepad

In [19]:
#sparkconf = SparkConf().setAppName("ODI").setMaster("local[3]")
#sc = SparkContext(conf=sparkconf)

In [20]:
captains_odi = sc.textFile( "C:/Users/Admin/Spark/Spark Practice/captains_ODI.csv",4,use_unicode=False).repartition(6)

In [21]:
captains_odi.take(10)

[b'Streak H H,Zimbabwe,1993-2005,68,18,47,3,29',
 b'Taylor M A,Australia,1989-1997,67,36,30,1,34',
 b'Atapattu M S,Sri Lanka,1990-2007,63,35,27,1,32',
 b'Javed Miandad,Pakistan,1975-1996,62,26,33,3,29',
 b'Strauss A J,England,2003-2011,62,27,33,2,31',
 b'Waqar Younis,Pakistan,1989-2003,62,37,23,2,33',
 b'Mendis L R D,Sri Lanka,1975-1989,61,11,46,4,29',
 b'Porterfield W T S*,Ireland,2006-,61,28,27,6,30',
 b'Howarth G P,New Zealand,1975-1985,60,31,26,3,23',
 b'Vaughan M P,England,2001-2007,60,32,22,6,28']

In [30]:
fields = ("Player", "Country", "Career", "Matches", "Won", "Lost", "Tied", "Toss" )
from collections import namedtuple
Captain = namedtuple( 'Captain', fields )
def parseRecs(line):
    fields = line.split(",")
    return Captain( fields[0], fields[1], fields[2], int(fields[3]), 
                   int(fields[4]), int(fields[5]), int(fields[6]), int(fields[7]))
Captains = captains_odi.map(lambda rec:parseRecs(rec))

In [1]:
Captains.take(10)

In [2]:
captains_100 = Captains.filter(lambda rec:rec.matches > 100)
#Captains.take(10)
captains_more_wins = captains_100.filter(lambda rec: rec.won > rec.lost)
countries = captains_more_wins.map(lambda rec:(rec.country, rec.matches))
matches_countries = countries.reduceByKey(lambda a, b: a + b)
matches_countries.collect()

In [3]:
captains_1001 = captains_100.filter(lambda rec:rec[1]>0)
captains_100_percent_wins = captains_1001.map(
    lambda rec:(rec.name, float(rec.won/rec.matches)))
result = captains_100_percent_wins.sortBy(lambda rec:rec[1], ascending=False).collect()

In [4]:
myList=[]
myList1=[]

for (a,b) in result:
    myList.append(a)
    myList1.append(b)
plt.pie(myList1, labels=myList)

plt.axis('equal')
plt.show()

In [5]:
counts=text_file.flatMap(lambda line: line.split("")).map(lambda word:(word,l)).reduceByKey(lambda a,b:a+b)